### Jul AdventKalender D19

https://adventofcode.com/2022/day/19

In [11]:
from mip import Model, xsum, maximize, BINARY
import re

In [16]:
mip.__version__

'1.14.1'

#### Day 19.1 

Given a set of blueprints that show the costs for building 4 types of robots:

    Blueprint 1:
      Each ore robot costs 4 ore.
      Each clay robot costs 2 ore.
      Each obsidian robot costs 3 ore and 14 clay.
      Each geode robot costs 2 ore and 7 obsidian.
   
Each robot can collect **1** of its resource type **per minute**. It also takes **one minute** to **construct** any type of robot, although it consumes the necessary resources available when construction **begins**. At the begining, there is exactly **one ore robot** uprunning. 

For each blueprint, figure out the **maximum** number of collected **geodes** after **24 minutes** by figuring out which robots to build and when to build them. Calculate the quality level of each blueprint by multiplying that blueprint's ID number with the largest number of geodes, and report the total quality level of all blueprints.

In [23]:
# ore robot: ore0; clay robot: ore1; obsidian robot: ore2,clay3; geode robot: ore4,obsidian5
def readFile(file_name):
    data_bp = []
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        costs = list(map(int, re.findall('[0-9]+', line)[1:]))
        data_bp.append(costs)
    f.close()
    return data_bp

In [24]:
def optimizer(mins, r_costs):
    n_robots = 4
    I = range(mins)
    m = Model("geode")

# optimize variables x:
# robot  min1  min2  min3  min4  ... min24
#   r1    1   binary, 0:not build 1:build
#   r2    0   binary, 0:not build 1:build
#   r3    0   binary, 0:not build 1:build
#   r4    0   binary, 0:not build 1:build

    x = [[m.add_var(var_type=BINARY) for i in I] for j in range(n_robots)] 
    m.objective = maximize(xsum((mins-i-1) * x[3][i] for i in I)) # goal, r4 produce geode
    
    # add constraints
    m += x[0][0] == 1 # start with 1 r1
    for i in I:
        m += (x[0][i]+x[1][i]+x[2][i]+x[3][i]) <=1 # can only build 1 robot per minute (building cost 1 minute)
    for i in range(1,mins-1):
        # for ore, clay and obsidian: the profit(until last minute) - the current build cost >= 0
        m += -xsum(x[0][j]*r_costs[0] for j in range(1,i+1)) - xsum(x[1][j]*r_costs[1] for j in range(i+1)) - xsum(x[2][j]*r_costs[2] for j in range(i+1)) - xsum(x[3][j]*r_costs[4] for j in range(i+1)) + xsum((i-j-1)*x[0][j] for j in range(i))+1 >= 0
        m += -xsum(x[2][j]*r_costs[3] for j in range(i+1)) + xsum((i-j-1)*x[1][j] for j in range(i)) >= 0
        m += -xsum(x[3][j]*r_costs[5] for j in range(i+1)) + xsum((i-j-1)*x[2][j] for j in range(i)) >= 0
    m.verbose = 0
    m.optimize()
    return sum((mins-1-i) * x[3][i].x for i in I)

In [26]:
data_bp = readFile('data/input19.txt')
minutes = 24
quality_level = 0
for i in range(len(data_bp)):
    max_geo = optimizer(minutes, data_bp[i])
    #print(i+1, data_bp[i], max_geo)
    quality_level += (i+1)*max_geo
quality_level

1147.0

#### Day 19.2

Given **32 minutes**, what are the maximum numbers of geodes for the **first 3 blueprints**? Calculate the multiplication of them.

In [27]:
minutes = 32
geo_mul = 1
for i in range(3):
    max_geo = optimizer(minutes, data_bp[i])
    geo_mul *= max_geo
geo_mul

3080.0